In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.\
        builder.\
        appName('lr_example').\
        getOrCreate()

## Preparing the data

In [3]:
data = spark.read.csv('Ecommerce_Customer.csv', header = True, inferSchema = True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(inputCols = data.columns[3:-1], outputCol = 'features')
output = assembler.transform(data)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [10]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [11]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                337|
|   mean| 503.97916253262883|
| stddev|  79.92459439866909|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [12]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                163|
|   mean|  489.6689653730027|
| stddev|  77.39112680719035|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



## Fitting and Predicting with Linear Regression Model

In [13]:
from pyspark.ml.regression import LinearRegression
model = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')
lr_model = model.fit(train_data)
test_results = lr_model.evaluate(test_data)

In [14]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  9.659637407710989|
|-13.615989479531322|
|-18.695572871577838|
|-5.8360553777928885|
|   5.42091713713512|
|-14.081296482314144|
|-23.467369782416995|
|  7.304381273979061|
| 1.4068646286522153|
| -9.380363313421356|
|-19.653191978227824|
|-3.0543923303372367|
|  16.86684517097433|
| -6.637731094990443|
|-27.452262311760137|
| -6.691950477024989|
| -3.022746755837886|
|  -8.22949356090129|
| 1.7916010694391957|
| -5.187036874246644|
+-------------------+
only showing top 20 rows



C:\apps\spark-3.2.1-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [15]:
# Root Mean Squared Error
test_results.rootMeanSquaredError

10.005851099948686

In [16]:
# R2 Score
test_results.r2

0.983181071615336

### Customers without labels

In [17]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.8162006488763...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.3091926408918...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.7656188210424...|
+--------------------+
only showing top 20 rows



In [18]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 398.9807136649165|
|[30.3931845423455...|333.54485928272493|
|[30.8162006488763...|284.78191382004684|
|[30.8794843441274...|496.04265536264757|
|[30.9716756438877...| 489.2176926197576|
|[31.0662181616375...| 463.0145896899885|
|[31.1239743499119...|510.41442362218277|
|[31.1695067987115...|420.05214952831375|
|[31.3091926408918...| 431.3138532112814|
|[31.5261978982398...|418.47488950575917|
|[31.5702008293202...| 565.5986841196327|
|[31.5761319713222...| 544.2809763196656|
|[31.6005122003032...| 462.3060063201226|
|[31.6253601348306...|382.97463185191464|
|[31.6739155032749...|503.17733022164134|
|[31.7207699002873...| 545.4668839550479|
|[31.7216523605090...| 350.7996733877105|
|[31.7242025238451...| 511.6173808488618|
|[31.7366356860502...| 495.1418451860927|
|[31.7656188210424...| 501.7411185098538|
+--------------------+------------